#### Code to train models

- Missing values inputation
- Avg for embed instead of cls
- Keep nan info as token
- Arcface
- Train on address only

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
cd ../src

## Imports

In [ ]:
import os
import ast
import glob
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
from tqdm.notebook import tqdm

In [ ]:
from params import *

from data.preparation import prepare_train_data, prepare_triplet_data
from data.dataset import TripletDataset
from data.tokenization import get_tokenizer

from model_zoo.models import SingleTransformer

from training.main import k_fold

from utils.logger import prepare_log_folder, save_config, create_logger

## Data

In [ ]:
df = prepare_train_data(root=DATA_PATH)

In [ ]:
triplets = prepare_triplet_data(root=DATA_PATH)

In [ ]:
triplets.head()

### Dataset

In [ ]:
name = "xlm-roberta-base"

tokenizer = get_tokenizer(name)

In [ ]:
dataset = TripletDataset(df, triplets, tokenizer)

In [ ]:
# lens = []
# for idx in tqdm(range(10000)):
#     data = dataset[idx]
#     lens += [len(data['ref_ids']), len(data['pos_ids']), len(data['neg_ids'])]

# sns.distplot(lens)

In [ ]:
idx = np.random.choice(len(dataset))
data = dataset[idx]

try:
    display(data['df'])
except:
    pass

## Model

In [ ]:
model = SingleTransformer(
    name, 
    nb_layers=1,
    embed_dim=128,
    nb_features=0,
).eval()

In [ ]:
ids = data["pos_ids"].unsqueeze(0)
fts = data["fts"][:, 0]

In [ ]:
representation, projection = model(ids, fts=fts)

In [ ]:
representation.size(), projection.size()

## Training

In [ ]:
BATCH_SIZES = {
    "xlm-roberta-base": 8,
    "xlm-roberta-large": 8,
    "roberta-large": 8,
}

LRS = {
    "xlm-roberta-base": 2e-5,
    "xlm-roberta-large": 1e-5,
    "roberta-large": 1e-5,
}

In [ ]:
class Config:
    # General
    seed = 2222
    device = "cuda"
    
    # Splits
    k = 2
    random_state = 2222
    selected_folds = [0]
    folds_file = DATA_PATH + "folds_2.csv"

    # Architecture
    name = "xlm-roberta-large"  # large ?
    pretrained_weights = None
    no_dropout = False
    nb_features = 0  # 2 for longitude, lattitude
    nb_layers = 1
    embed_dim = 384  # 256

    # Texts
    max_len = 100
    lower = False  # TODO

    # Training    
    loss_config = {
        "margin": 1.,
        "p": 2.,
    }

    data_config = {
        "batch_size": BATCH_SIZES[name],
        "val_bs": BATCH_SIZES[name] * 2,
        "use_len_sampler": False,
        "pad_token": 1 if "roberta" in name else 0,
    }

    optimizer_config = {
        "name": "AdamW",
        "lr": LRS[name],
        "lr_transfo": LRS[name],
        "lr_decay": 1,
        "warmup_prop": 0.1,
        "weight_decay": 1,
        "betas": (0.5, 0.99),
        "max_grad_norm": 1.,
    }

    gradient_checkpointing = False
    acc_steps = 1
    epochs = 1

    use_fp16 = True

    verbose = 1
    verbose_eval = 1000

In [ ]:
DEBUG = False
log_folder = None

In [ ]:
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    print(f'Logging results to {log_folder}')
    save_config(Config, log_folder + 'config')
    df.to_csv(log_folder + 'data.csv', index=False)
    create_logger(directory=log_folder, name="logs.txt")

k_fold(
    Config,
    df,
    triplets,
    log_folder=log_folder
)

Done ! 